In [ ]:
import json

In [4]:
import json
from pathlib import Path

def calculate_overlap_metric(annotation_file, prediction_file):
    # JSONファイルを読み込む
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    with open(prediction_file, "r") as f:
        predictions = json.load(f)

    results = {}

    for annotation in annotations:
        video_name = annotation["name"]
        if video_name not in predictions:
            continue

        # 正解データからpitchの範囲を取得
        pitch_ranges = annotation.get("attributes", [])
        pitch_ranges = [attr for attr in pitch_ranges if attr["key"] == "pitch"]
        if not pitch_ranges:
            continue
        pitch_ranges = pitch_ranges[0]["value"]

        # 予測データを取得し、512フレームずつのチャンクをフラット化
        predicted_chunks = predictions[video_name]
        predicted_frames = [frame for chunk in predicted_chunks for frame in chunk]

        total_overlap = 0
        total_union = 0

        for pitch_range in pitch_ranges:
            T0, T1 = pitch_range

            # 予測範囲で1になっているインデックスを取得
            t0 = next((i for i, x in enumerate(predicted_frames) if x == 1 and i >= T0), None)
            t1 = next((i for i, x in enumerate(predicted_frames[::-1]) if x == 1 and len(predicted_frames) - i - 1 <= T1), None)
            t1 = len(predicted_frames) - t1 - 1 if t1 is not None else None

            if t0 is not None and t1 is not None:
                # ユニオンの計算（開始点と終了点）
                union_start = min(T0, t0)
                union_end = max(T1, t1)
                union = union_end - union_start + 1

                # オーバーラップの計算
                overlap_start = max(T0, t0)
                overlap_end = min(T1, t1)
                overlap = max(0, overlap_end - overlap_start + 1)

                total_overlap += overlap
                total_union += union

                print(f"Video: {video_name}, Pitch Range: ({T0}, {T1}), Prediction Range: ({t0}, {t1}), Overlap: {overlap}, Union: {union}")

        # 結果を保存（分子と分母も含める）
        results[video_name] = {
            "overlap": total_overlap,
            "union": total_union,
            "score": total_overlap / total_union if total_union > 0 else 0.0
        }

    return results

if __name__ == "__main__":
    annotation_file = "/home/ubuntu/slocal/PitchDetection/json/test_annotations.json"
    prediction_file = "/home/ubuntu/slocal/PitchDetection/test_predictions.json"

    metrics = calculate_overlap_metric(annotation_file, prediction_file)

    for video, data in metrics.items():
        print(f"{video}: overlap = {data['overlap']}, union = {data['union']}, score = {data['score']:.4f}")


Video: test_HvsW.mp4, Pitch Range: (334, 488), Prediction Range: (334, 488), Overlap: 155, Union: 155
Video: test_HvsW.mp4, Pitch Range: (715, 916), Prediction Range: (715, 882), Overlap: 168, Union: 202
Video: test_HvsW.mp4, Pitch Range: (1919, 2042), Prediction Range: (1919, 2042), Overlap: 124, Union: 124
Video: test_HvsW.mp4, Pitch Range: (2766, 2931), Prediction Range: (2766, 2913), Overlap: 148, Union: 166
Video: test_HvsW.mp4, Pitch Range: (4059, 4203), Prediction Range: (4096, 4188), Overlap: 93, Union: 145
Video: test_HvsW.mp4, Pitch Range: (5429, 5568), Prediction Range: (5429, 5549), Overlap: 121, Union: 140
Video: test_HvsW.mp4, Pitch Range: (6056, 6219), Prediction Range: (6056, 6143), Overlap: 88, Union: 164
Video: test_KvsW.mp4, Pitch Range: (213, 363), Prediction Range: (213, 363), Overlap: 151, Union: 151
Video: test_KvsW.mp4, Pitch Range: (539, 677), Prediction Range: (539, 633), Overlap: 95, Union: 139
Video: test_KvsW.mp4, Pitch Range: (838, 970), Prediction Range: 

In [5]:
import json
from pathlib import Path

def calculate_overlap_metric(annotation_file, prediction_file):
    # JSONファイルを読み込む
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    with open(prediction_file, "r") as f:
        predictions = json.load(f)

    results = {}

    for annotation in annotations:
        video_name = annotation["name"]
        if video_name not in predictions:
            continue

        # 正解データからpitchの範囲を取得
        pitch_ranges = annotation.get("attributes", [])
        pitch_ranges = [attr for attr in pitch_ranges if attr["key"] == "pitch"]
        if not pitch_ranges:
            continue
        pitch_ranges = pitch_ranges[0]["value"]

        # 予測データを取得し、512フレームずつのチャンクをフラット化
        predicted_chunks = predictions[video_name]
        predicted_frames = [frame for chunk in predicted_chunks for frame in chunk]

        total_overlap = 0
        total_union = 0

        for pitch_range in pitch_ranges:
            T0, T1 = pitch_range

            # チャンク内のオフセットに基づく計算
            chunk_size = 512
            start_chunk = T0 // chunk_size
            end_chunk = T1 // chunk_size

            # 予測範囲を計算
            t0 = None
            t1 = None
            for chunk_idx in range(start_chunk, end_chunk + 1):
                if chunk_idx < len(predicted_chunks):
                    chunk = predicted_chunks[chunk_idx]
                    offset = chunk_idx * chunk_size

                    for i, value in enumerate(chunk):
                        if value == 1:
                            global_idx = offset + i
                            if t0 is None or global_idx < t0:
                                t0 = global_idx
                            if global_idx > (t1 or 0):
                                t1 = global_idx

            if t0 is not None and t1 is not None:
                # ユニオンの計算（開始点と終了点）
                union_start = min(T0, t0)
                union_end = max(T1, t1)
                union = union_end - union_start + 1

                # オーバーラップの計算
                overlap_start = max(T0, t0)
                overlap_end = min(T1, t1)
                overlap = max(0, overlap_end - overlap_start + 1)

                total_overlap += overlap
                total_union += union

                print(f"Video: {video_name}, Pitch Range: ({T0}, {T1}), Prediction Range: ({t0}, {t1}), Overlap: {overlap}, Union: {union}")

        # 結果を保存（分子と分母も含める）
        results[video_name] = {
            "overlap": total_overlap,
            "union": total_union,
            "score": total_overlap / total_union if total_union > 0 else 0.0
        }

    return results

if __name__ == "__main__":
    annotation_file = "/home/ubuntu/slocal/PitchDetection/json/test_annotations.json"
    prediction_file = "/home/ubuntu/slocal/PitchDetection/test_predictions.json"

    metrics = calculate_overlap_metric(annotation_file, prediction_file)

    for video, data in metrics.items():
        print(f"{video}: overlap = {data['overlap']}, union = {data['union']}, score = {data['score']:.4f}")


Video: test_HvsW.mp4, Pitch Range: (334, 488), Prediction Range: (330, 511), Overlap: 155, Union: 182
Video: test_HvsW.mp4, Pitch Range: (715, 916), Prediction Range: (705, 882), Overlap: 168, Union: 212
Video: test_HvsW.mp4, Pitch Range: (1919, 2042), Prediction Range: (1887, 2047), Overlap: 124, Union: 161
Video: test_HvsW.mp4, Pitch Range: (2766, 2931), Prediction Range: (2730, 2913), Overlap: 148, Union: 202
Video: test_HvsW.mp4, Pitch Range: (4059, 4203), Prediction Range: (4096, 4188), Overlap: 93, Union: 145
Video: test_HvsW.mp4, Pitch Range: (5429, 5568), Prediction Range: (5389, 5549), Overlap: 121, Union: 180
Video: test_HvsW.mp4, Pitch Range: (6056, 6219), Prediction Range: (6039, 6143), Overlap: 88, Union: 181
Video: test_KvsW.mp4, Pitch Range: (213, 363), Prediction Range: (209, 382), Overlap: 151, Union: 174
Video: test_KvsW.mp4, Pitch Range: (539, 677), Prediction Range: (512, 967), Overlap: 139, Union: 456
Video: test_KvsW.mp4, Pitch Range: (838, 970), Prediction Range:

In [6]:
import json
from pathlib import Path

def calculate_overlap_metric(annotation_file, prediction_file):
    # JSONファイルを読み込む
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    with open(prediction_file, "r") as f:
        predictions = json.load(f)

    results = {}

    for annotation in annotations:
        video_name = annotation["name"]
        if video_name not in predictions:
            continue

        # 正解データからpitchの範囲を取得
        pitch_ranges = annotation.get("attributes", [])
        pitch_ranges = [attr for attr in pitch_ranges if attr["key"] == "pitch"]
        if not pitch_ranges:
            continue
        pitch_ranges = pitch_ranges[0]["value"]

        # 予測データを取得し、512フレームずつのチャンクをフラット化
        predicted_chunks = predictions[video_name]
        predicted_frames = [frame for chunk in predicted_chunks for frame in chunk]

        total_overlap = 0
        total_union = 0
        pitch_count = len(pitch_ranges)

        for pitch_range in pitch_ranges:
            T0, T1 = pitch_range

            # チャンク内のオフセットに基づく計算
            chunk_size = 512
            start_chunk = T0 // chunk_size
            end_chunk = T1 // chunk_size

            # 予測範囲を計算
            t0 = None
            t1 = None
            for chunk_idx in range(start_chunk, end_chunk + 1):
                if chunk_idx < len(predicted_chunks):
                    chunk = predicted_chunks[chunk_idx]
                    offset = chunk_idx * chunk_size

                    for i, value in enumerate(chunk):
                        if value == 1:
                            global_idx = offset + i
                            if t0 is None or global_idx < t0:
                                t0 = global_idx
                            if global_idx > (t1 or 0):
                                t1 = global_idx

            if t0 is not None and t1 is not None:
                # ユニオンの計算（開始点と終了点）
                union_start = min(T0, t0)
                union_end = max(T1, t1)
                union = union_end - union_start + 1

                # オーバーラップの計算
                overlap_start = max(T0, t0)
                overlap_end = min(T1, t1)
                overlap = max(0, overlap_end - overlap_start + 1)

                total_overlap += overlap
                total_union += union

        # 結果を保存（分子と分母も含める）
        results[video_name] = {
            "pitch_count": pitch_count,
            "overlap": total_overlap,
            "union": total_union,
            "score": total_overlap / total_union if total_union > 0 else 0.0
        }

    return results

if __name__ == "__main__":
    annotation_file = "/home/ubuntu/slocal/PitchDetection/json/test_annotations.json"
    prediction_file = "/home/ubuntu/slocal/PitchDetection/test_predictions.json"

    metrics = calculate_overlap_metric(annotation_file, prediction_file)

    final_score = 0
    total_pitches = 0

    for video, data in metrics.items():
        print(f"{video}: pitch_count = {data['pitch_count']}, score = {data['score']:.4f}")
        final_score += data['score'] * data['pitch_count']
        total_pitches += data['pitch_count']

    final_score /= total_pitches if total_pitches > 0 else 1
    print(f"Final Score: {final_score:.4f}")


test_HvsW.mp4: pitch_count = 7, score = 0.7102
test_KvsW.mp4: pitch_count = 6, score = 0.2924
test_tachikawa.mp4: pitch_count = 7, score = 0.5652
test_Yasuda.mp4: pitch_count = 8, score = 0.4012
test_Yokosuka.mp4: pitch_count = 7, score = 0.6030
Final Score: 0.5175


test_HvsW.mp4: pitch_count = 7, score = 0.7102
test_KvsW.mp4: pitch_count = 6, score = 0.2924
test_tachikawa.mp4: pitch_count = 7, score = 0.5652
test_Yasuda.mp4: pitch_count = 8, score = 0.4012
test_Yokosuka.mp4: pitch_count = 7, score = 0.6030
Final Score: 0.5175

In [8]:
import json
from pathlib import Path

def calculate_overlap_metric(annotation_file, prediction_file):
    # JSONファイルを読み込む
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    with open(prediction_file, "r") as f:
        predictions = json.load(f)

    results = {}

    for annotation in annotations:
        video_name = annotation["name"]
        if video_name not in predictions:
            continue

        # 正解データからpitchの範囲を取得
        pitch_ranges = annotation.get("attributes", [])
        pitch_ranges = [attr for attr in pitch_ranges if attr["key"] == "pitch"]
        if not pitch_ranges:
            continue
        pitch_ranges = pitch_ranges[0]["value"]

        # 予測データを取得し、512フレームずつのチャンクをフラット化
        predicted_chunks = predictions[video_name]
        predicted_frames = [frame for chunk in predicted_chunks for frame in chunk]

        total_overlap = 0
        total_union = 0
        pitch_count = len(pitch_ranges)

        for pitch_range in pitch_ranges:
            T0, T1 = pitch_range

            # チャンク内のオフセットに基づく計算
            chunk_size = 512
            start_chunk = T0 // chunk_size
            end_chunk = T1 // chunk_size

            # 予測範囲を計算
            t0 = None
            t1 = None
            for chunk_idx in range(start_chunk, end_chunk + 1):
                if chunk_idx < len(predicted_chunks):
                    chunk = predicted_chunks[chunk_idx]
                    offset = chunk_idx * chunk_size

                    for i, value in enumerate(chunk):
                        if value == 1:
                            global_idx = offset + i
                            if t0 is None or global_idx < t0:
                                t0 = global_idx
                            if global_idx > (t1 or 0):
                                t1 = global_idx

            if t0 is not None and t1 is not None and t1 >= t0:
                # ユニオンの計算（開始点と終了点）
                union_start = min(T0, t0)
                union_end = max(T1, t1)
                union = union_end - union_start + 1

                # オーバーラップの計算
                overlap_start = max(T0, t0)
                overlap_end = min(T1, t1)
                overlap = max(0, overlap_end - overlap_start + 1)

                if union > 0:
                    total_overlap += overlap
                    total_union += union

                    print(f"Video: {video_name}, Pitch Range: ({T0}, {T1}), Prediction Range: ({t0}, {t1}), Overlap: {overlap}, Union: {union}, Score: {overlap / union:.4f}")

        # 結果を保存（分子と分母も含める）
        if total_union > 0:
            results[video_name] = {
                "pitch_count": pitch_count,
                "overlap": total_overlap,
                "union": total_union,
                "score": total_overlap / total_union
            }

    return results

if __name__ == "__main__":
    annotation_file = "/home/ubuntu/slocal/PitchDetection/json/test_annotations.json"
    prediction_file = "/home/ubuntu/slocal/PitchDetection/test_predictions.json"

    metrics = calculate_overlap_metric(annotation_file, prediction_file)

    final_score = 0
    total_pitches = 0

    for video, data in metrics.items():
        print(f"{video}: pitch_count = {data['pitch_count']}, score = {data['score']:.4f}")
        final_score += data['score'] * data['pitch_count']
        total_pitches += data['pitch_count']

    final_score /= total_pitches if total_pitches > 0 else 1
    print(f"Final Score: {final_score:.4f}")


Video: test_HvsW.mp4, Pitch Range: (334, 488), Prediction Range: (330, 511), Overlap: 155, Union: 182, Score: 0.8516
Video: test_HvsW.mp4, Pitch Range: (715, 916), Prediction Range: (705, 882), Overlap: 168, Union: 212, Score: 0.7925
Video: test_HvsW.mp4, Pitch Range: (1919, 2042), Prediction Range: (1887, 2047), Overlap: 124, Union: 161, Score: 0.7702
Video: test_HvsW.mp4, Pitch Range: (2766, 2931), Prediction Range: (2730, 2913), Overlap: 148, Union: 202, Score: 0.7327
Video: test_HvsW.mp4, Pitch Range: (4059, 4203), Prediction Range: (4096, 4188), Overlap: 93, Union: 145, Score: 0.6414
Video: test_HvsW.mp4, Pitch Range: (5429, 5568), Prediction Range: (5389, 5549), Overlap: 121, Union: 180, Score: 0.6722
Video: test_HvsW.mp4, Pitch Range: (6056, 6219), Prediction Range: (6039, 6143), Overlap: 88, Union: 181, Score: 0.4862
Video: test_KvsW.mp4, Pitch Range: (213, 363), Prediction Range: (209, 382), Overlap: 151, Union: 174, Score: 0.8678
Video: test_KvsW.mp4, Pitch Range: (539, 677),

In [9]:
import json
from pathlib import Path

def calculate_overlap_metric(annotation_file, prediction_file):
    # JSONファイルを読み込む
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    with open(prediction_file, "r") as f:
        predictions = json.load(f)

    results = {}

    for annotation in annotations:
        video_name = annotation["name"]
        if video_name == "test_tachikawa.mp4":
            continue  # tachikawaの動画をスキップ

        if video_name not in predictions:
            continue

        # 正解データからpitchの範囲を取得
        pitch_ranges = annotation.get("attributes", [])
        pitch_ranges = [attr for attr in pitch_ranges if attr["key"] == "pitch"]
        if not pitch_ranges:
            continue
        pitch_ranges = pitch_ranges[0]["value"]

        # 予測データを取得し、512フレームずつのチャンクをフラット化
        predicted_chunks = predictions[video_name]
        predicted_frames = [frame for chunk in predicted_chunks for frame in chunk]

        total_overlap = 0
        total_union = 0
        pitch_count = len(pitch_ranges)

        for pitch_range in pitch_ranges:
            T0, T1 = pitch_range

            # チャンク内のオフセットに基づく計算
            chunk_size = 512
            start_chunk = T0 // chunk_size
            end_chunk = T1 // chunk_size

            # 予測範囲を計算
            t0 = None
            t1 = None
            for chunk_idx in range(start_chunk, end_chunk + 1):
                if chunk_idx < len(predicted_chunks):
                    chunk = predicted_chunks[chunk_idx]
                    offset = chunk_idx * chunk_size

                    for i, value in enumerate(chunk):
                        if value == 1:
                            global_idx = offset + i
                            if t0 is None or global_idx < t0:
                                t0 = global_idx
                            if global_idx > (t1 or 0):
                                t1 = global_idx

            if t0 is not None and t1 is not None and t1 >= t0:
                # ユニオンの計算（開始点と終了点）
                union_start = min(T0, t0)
                union_end = max(T1, t1)
                union = union_end - union_start + 1

                # オーバーラップの計算
                overlap_start = max(T0, t0)
                overlap_end = min(T1, t1)
                overlap = max(0, overlap_end - overlap_start + 1)

                if union > 0:
                    total_overlap += overlap
                    total_union += union

                    print(f"Video: {video_name}, Pitch Range: ({T0}, {T1}), Prediction Range: ({t0}, {t1}), Overlap: {overlap}, Union: {union}, Score: {overlap / union:.4f}")

        # 結果を保存（分子と分母も含める）
        if total_union > 0:
            results[video_name] = {
                "pitch_count": pitch_count,
                "overlap": total_overlap,
                "union": total_union,
                "score": total_overlap / total_union
            }

    return results

if __name__ == "__main__":
    annotation_file = "/home/ubuntu/slocal/PitchDetection/json/test_annotations.json"
    prediction_file = "/home/ubuntu/slocal/PitchDetection/test_predictions.json"

    metrics = calculate_overlap_metric(annotation_file, prediction_file)

    final_score = 0
    total_pitches = 0

    for video, data in metrics.items():
        print(f"{video}: pitch_count = {data['pitch_count']}, score = {data['score']:.4f}")
        final_score += data['score'] * data['pitch_count']
        total_pitches += data['pitch_count']

    final_score /= total_pitches if total_pitches > 0 else 1
    print(f"Final Score: {final_score:.4f}")


Video: test_HvsW.mp4, Pitch Range: (334, 488), Prediction Range: (330, 511), Overlap: 155, Union: 182, Score: 0.8516
Video: test_HvsW.mp4, Pitch Range: (715, 916), Prediction Range: (705, 882), Overlap: 168, Union: 212, Score: 0.7925
Video: test_HvsW.mp4, Pitch Range: (1919, 2042), Prediction Range: (1887, 2047), Overlap: 124, Union: 161, Score: 0.7702
Video: test_HvsW.mp4, Pitch Range: (2766, 2931), Prediction Range: (2730, 2913), Overlap: 148, Union: 202, Score: 0.7327
Video: test_HvsW.mp4, Pitch Range: (4059, 4203), Prediction Range: (4096, 4188), Overlap: 93, Union: 145, Score: 0.6414
Video: test_HvsW.mp4, Pitch Range: (5429, 5568), Prediction Range: (5389, 5549), Overlap: 121, Union: 180, Score: 0.6722
Video: test_HvsW.mp4, Pitch Range: (6056, 6219), Prediction Range: (6039, 6143), Overlap: 88, Union: 181, Score: 0.4862
Video: test_KvsW.mp4, Pitch Range: (213, 363), Prediction Range: (209, 382), Overlap: 151, Union: 174, Score: 0.8678
Video: test_KvsW.mp4, Pitch Range: (539, 677),